## Accuracy Score

In [1]:
import numpy as np
from sklearn.base import BaseEstimator

# scikit learn의 Classifier 구현을 흉내 : Estimator들은 BaseEstimator를 상속받아야 GridSearchCV, cross_val_score()등 다양한 utility class들과 자연스럽게 결합이 가능함!

class MyDummyClassifier (BaseEstimator):
    # fit() method는 아무것도 학습 안하는걸로 정의
    def fit(self, X, y=None):
        pass
    # predict() method는 단순히 Sex feature가 1(남자)면 0(사망), 그렇지 않으면 1(생존)으로 예측하도록 정의
    def predict(self, X):
        pred = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
                
        return pred
            

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터를 재로딩, 데이터 가공, 학습데이터/테스트 데이터 분할. 
titanic_df = pd.read_csv('train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, \
                                                  test_size=0.2, random_state=0)

위에서 생성한 Dummy Classifier를 이용하여 학습-예측-평가 수행

In [5]:
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypred = myclf.predict(X_test)
print('My Dummy Classifier의 정확도는:', accuracy_score(y_test, mypred))

My Dummy Classifier의 정확도는: 0.7877094972067039


##### 단순하게 남성은 사망, 여성은 생존으로 예측하게 만들었는데도 'accuracy'로 평가를 해보니, 정확도가 무려 78%가 나온다. (이진 분류의 경우 문제 있음)

In [10]:
from sklearn.datasets import load_digits

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass

    # 입력값으로 들어오는 X 데이터 셋의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X):
        return np.zeros( (len(X), 1) , dtype=bool )

# 사이킷런의 내장 데이터 셋인 load_digits( )를 이용하여 MNIST 데이터 로딩 
digits = load_digits()

print(digits.data)
print("### digits.data.shape:", digits.data.shape)
print(digits.target)
print("### digits.target.shape:", digits.target.shape)

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
### digits.data.shape: (1797, 64)
[0 1 2 ... 8 9 8]
### digits.target.shape: (1797,)


In [11]:
# 현재 digit.target이 7로 되어 있는 수는?
(digits.target == 7).sum()

179

In [13]:
# digits 번호가 7이면 'True', 1로 반환 / 7이 아니면 'False', 0으로 반환
y = (digits.target == 7).astype(int)
Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, y, random_state = 11)

In [14]:
# 불균형한 레이블 데이터 분포도 확인. 
print('레이블 테스트 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0 과 1의 분포도')
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train , y_train)
fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracy_score(y_test , fakepred)))

레이블 테스트 세트 크기 : (179,)
테스트 세트 레이블 0 과 1의 분포도
0    110
1     69
Name: Survived, dtype: int64
모든 예측을 0으로 하여도 정확도는:0.615


이와 같이 target의 분포가 불균형한 분포를 가질 때 accuracy score가 한계가 있음을 알 수 있다. 따라서, accuracy 말고 다른 평가 지표를 알아두자. 

## Confusion Matrix

**오차행렬(confusion matrix):** 이진분류의 예측 오류가 얼마인지와 더불어 어떠한 유형의 예측 오류가 발생하고 있는지를 함께 나타내는 지표

In [15]:
from sklearn.metrics import confusion_matrix

# 앞에서 예측한 결과인 fakepred와 실제 결과인 y_test의 Confusion Matrix출력
confusion_matrix(y_test , fakepred)

array([[110,   0],
       [ 69,   0]], dtype=int64)

## Precision(정밀도) & Recall(재현율)